# Visualize any Graph

Load and visualize a hierarchical/normal graph generated by using anemoi-graph HierarchicalGraphCreator or GraphCreator.

## Load the Graph

In [ ]:
import torch
from anemoi.graphs.plotting.interactive.graph_3d import plot_downscale, plot_upscale, plot_level

In [ ]:
# Specify path to graph
path = 'path_to_graph.pt'
num_hidden = 1

# Load graph and separate each sub-graph
hetero_data = torch.load(path, weights_only=False) 
data_nodes = hetero_data['data'].x
hidden_nodes = []
hidden_edges = []
downscale_edges = []
upscale_edges = []

if num_hidden > 1:

    data_to_hidden_edges = hetero_data[('data', 'to', 'hidden_1')].edge_index

    for i in range(1, num_hidden):
        hidden_nodes.append(hetero_data[f'hidden_{i}'].x)
        hidden_edges.append(hetero_data[(f'hidden_{i}', 'to', f'hidden_{i}')].edge_index)
        downscale_edges.append(hetero_data[(f'hidden_{i}', 'to', f'hidden_{i+1}')].edge_index)
        upscale_edges.append(hetero_data[(f'hidden_{num_hidden+1-i}', 'to', f'hidden_{num_hidden-i}')].edge_index)

    # Add hidden-most layer
    hidden_nodes.append(hetero_data[f'hidden_{num_hidden}'].x) 
    hidden_edges.append(hetero_data[(f'hidden_{num_hidden}', 'to', f'hidden_{num_hidden}')].edge_index)
    # Add symbolic graphs for last layers of downscaling and upscaling -> they do not have edges
    downscale_edges.append(hetero_data[(f'hidden_{num_hidden}', 'to', f'hidden_{i}')].edge_index)
    upscale_edges.append(hetero_data[('hidden_1', 'to', 'data')].edge_index)

    hidden_to_data_edges = hetero_data[('hidden_1', 'to', 'data')].edge_index

else:
    try:
            data_to_hidden_edges = hetero_data[('data', 'to', 'hidden_1')].edge_index
            hidden_nodes.append(hetero_data['hidden_1'].x)
            hidden_edges.append(hetero_data[('hidden_1', 'to', 'hidden_1')].edge_index)
            downscale_edges.append(hetero_data[('data', 'to', 'hidden_1')].edge_index)
            upscale_edges.append(hetero_data[('hidden_1', 'to', 'data')].edge_index)
            hidden_to_data_edges = hetero_data[('hidden_1', 'to', 'data')].edge_index
        
    except Exception:
        data_to_hidden_edges = hetero_data[('data', 'to', 'hidden')].edge_index
        hidden_nodes.append(hetero_data['hidden'].x)
        hidden_edges.append(hetero_data[('hidden', 'to', 'hidden')].edge_index)
        downscale_edges.append(hetero_data[('data', 'to', 'hidden')].edge_index)
        upscale_edges.append(hetero_data[('hidden', 'to', 'data')].edge_index)
        hidden_to_data_edges = hetero_data[('hidden', 'to', 'data')].edge_index

print(f'Lat Lon grid has: {len(data_nodes)} points.')
for i in range(num_hidden):
    print(f'Hidden layer {i+1} has: {len(hidden_nodes[i])} points')

## Encoder + Downscaling

In [ ]:
fig = plot_downscale(data_nodes, hidden_nodes, data_to_hidden_edges, downscale_edges, title='Downscaling',  color='red', num_hidden=num_hidden, x_range=[0, 0,4], y_range=[0, 0.4], z_range=[0, 0.4])
fig.show()

## Upscaling + Decoder


In [ ]:
fig = plot_upscale(data_nodes, hidden_nodes, hidden_to_data_edges, upscale_edges, title='Upscaling', color='blue', num_hidden=num_hidden,x_range=[0, 0,4], y_range=[0, 0.4], z_range=[0, 0.4])
fig.show()

## Same Level Processing

In [ ]:
fig = plot_level(data_nodes, hidden_nodes, data_to_hidden_edges, hidden_edges, title='Level Processing', color='green', num_hidden=num_hidden, x_range=[0, 0,4], y_range=[0, 0.4], z_range=[0, 0.4])
fig.show()